In [3]:
import pandas as pd
import plotly.express as px
import json
from firebase import firebase

In [4]:
#Tutorial we followed for reading data in firebase from python: https://morioh.com/p/4dca3ded4cea

firebase = firebase.FirebaseApplication('https://safer-walks-default-rtdb.firebaseio.com/', None)
parents_data = firebase.get('/RegisteredParents', '')

In [5]:
#reading schoolsdata file which contains all schools in Sweden.
schools_file = 'schoolsdata.csv'
df_schools = pd.read_csv(schools_file)
df_schools

,school_long,school_lat,osm_id,school_name
0,11.939127,57.694679,4742340,Fjällskolan
1,11.945730,57.696740,4768432,Oscar Fredriksskolan
2,18.116401,59.382723,4875003,Sticklinge skola
3,17.914796,59.482734,6062883,Rotsunda skola
4,17.920795,59.465465,13862875,Stallets skola
...,...,...,...,...
3105,14.499669,59.316717,871653648,Skrantaskolan
3106,14.077498,57.894733,871990590,I Ur och Skur Oxdragaren
3107,14.082188,57.919182,872880398,Förskolan Äventyret
3108,13.931429,55.456706,874056551,Köpingebro skola


In [6]:
#Creating a dataframe from parents_data. 
df_parents = pd.DataFrame()
for i, parent in enumerate(parents_data.values()):
    df_parents = df_parents.append(pd.DataFrame(parent, index=[i]), ignore_index=True)

In [7]:
#Splitting userLocation from df_parents, into latitude: user_lat and longitude: user_long.
df_parents['user_lat'] = [float(s.split(',')[0]) for s in df_parents['userLocation']]
df_parents['user_long'] = [float(s.split(',')[1]) for s in df_parents['userLocation']]

In [8]:
#Merging df_schools into df_parents by schoolId(df_parents) and osm_id(df_schools)
df = pd.merge(df_parents, df_schools, left_on='schoolId', right_on='osm_id')

In [9]:
df.groupby("school_name").count()

,parentName,schoolId,timeRegistration,userLocation,user_lat,user_long,school_long,school_lat,osm_id
school_name,,,,,,,,,
Edsbergsskolan,8,8,8,8,8,8,8,8,8
Gärdeskolan,8,8,8,8,8,8,8,8,8
Hagalundsskolan,2,2,2,2,2,2,2,2,2
Kärrdalsskolan,5,5,5,5,5,5,5,5,5
Nordiska musikgymnasiet,2,2,2,2,2,2,2,2,2
Pärlan,2,2,2,2,2,2,2,2,2
Rudbeck Valfrihetens Gymnasium,7,7,7,7,7,7,7,7,7
Råsunda skola,3,3,3,3,3,3,3,3,3
Rösjöskolan,9,9,9,9,9,9,9,9,9


In [10]:
token = open(".mapbox_token").read() #This file must not go in the commit in git.
#Creating the map
px.set_mapbox_access_token(token)
df["schoolId"] = df["schoolId"].astype(str)
fig = px.scatter_mapbox(df, lon='user_long', lat='user_lat', color='school_name',
                        hover_data=['parentName', 'timeRegistration'])
fig.update_layout(
    mapbox = {
        'style': "satellite"},
    showlegend = True)
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '.mapbox_token'

In [11]:
df


,parentName,schoolId,timeRegistration,userLocation,user_lat,user_long,school_long,school_lat,osm_id,school_name
0,Johan Eriksson,299737854,"24, Dec, 2020, 12:05","59.2794315,17.8900085",59.279432,17.890009,17.899835,59.273127,299737854,Västerholms friskola
1,Sofia Andersson,299737854,"24, Dec, 2020, 14:05","59.2794355,17.8899913",59.279435,17.889991,17.899835,59.273127,299737854,Västerholms friskola
2,Manpreet Singh,299737854,"24, Dec, 2020, 15:28","59.2796401,17.8918366",59.279640,17.891837,17.899835,59.273127,299737854,Västerholms friskola
3,Petra Johansson,299737854,"24, Dec, 2020, 15:36","59.2779426,17.8854605",59.277943,17.885461,17.899835,59.273127,299737854,Västerholms friskola
4,Ana Varela,70223549,"24, Dec, 2020, 12:12","59.4441698,17.9733782",59.444170,17.973378,17.976933,59.446505,70223549,Rösjöskolan
5,Camilo Lopez,70223549,"24, Dec, 2020, 12:15","59.4441556,17.9734932",59.444156,17.973493,17.976933,59.446505,70223549,Rösjöskolan
6,Patricia King,70223549,"24, Dec, 2020, 12:19","59.4447571,17.9739011",59.444757,17.973901,17.976933,59.446505,70223549,Rösjöskolan
7,Charles Monaghan,70223549,"24, Dec, 2020, 12:19","59.44476,17.9736518",59.444760,17.973652,17.976933,59.446505,70223549,Rösjöskolan
8,Jhon Botts,70223549,"24, Dec, 2020, 12:24","59.4455232,17.9744158",59.445523,17.974416,17.976933,59.446505,70223549,Rösjöskolan
9,Ricky Kwon,70223549,"24, Dec, 2020, 12:51","59.447481,17.9684811",59.447481,17.968481,17.976933,59.446505,70223549,Rösjöskolan


In [12]:
df.groupby("school_name").count()

,parentName,schoolId,timeRegistration,userLocation,user_lat,user_long,school_long,school_lat,osm_id
school_name,,,,,,,,,
Edsbergsskolan,8,8,8,8,8,8,8,8,8
Gärdeskolan,8,8,8,8,8,8,8,8,8
Hagalundsskolan,2,2,2,2,2,2,2,2,2
Kärrdalsskolan,5,5,5,5,5,5,5,5,5
Nordiska musikgymnasiet,2,2,2,2,2,2,2,2,2
Pärlan,2,2,2,2,2,2,2,2,2
Rudbeck Valfrihetens Gymnasium,7,7,7,7,7,7,7,7,7
Råsunda skola,3,3,3,3,3,3,3,3,3
Rösjöskolan,9,9,9,9,9,9,9,9,9


In [13]:
dff = df.loc[df["schoolId"] == 70223549 ]

In [14]:
numerical = ["user_long", "user_lat"]

X = dff[[c for c in dff.columns if c in numerical]]


X = pd.DataFrame(X, columns=numerical)
y = dff.iloc[:, -1]

In [15]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [16]:

kmeans = KMeans(n_clusters=3)

#compute kmeans clustering
kmeans.fit(X)
#call the kmeans object, predict, to predict the cluster index of each sample. 

y_kmeans = kmeans.predict(X)

In [17]:
y_kmeans

array([1, 1, 1, 1, 1, 2, 2, 2, 0], dtype=int32)

In [18]:
def evaluation_metrics(X, labels_pred, metric_name):


        s_s = metrics.silhouette_score(X, labels_pred, metric=metric_name)
       

        metrics_names = ["Silh_S"]
        values = [s_s]
        

        result = list(zip(metrics_names,values))
        result = pd.DataFrame(result, columns=['Metric','Value'])

        return result

In [19]:
evaluation_metrics(X, kmeans.labels_, "euclidean" )

,Metric,Value
0,Silh_S,0.606623


In [26]:
import hdbscan

In [43]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, metric = 'haversine')
clusterer.fit(X)

HDBSCAN(metric='haversine', min_cluster_size=3)

In [44]:
hdbscan_result = clusterer.labels_

In [45]:
hdbscan_result

array([0, 0, 0, 0, 0, 1, 1, 1, 1])